In [ ]:
import cv2
from deepface import DeepFace
import os # Import the os module for path manipulation

# Load your reference image of your dad
reference_img_path = "videos/me.jpeg" # Ensure this path is correct and the image is accessible

# Load video file (or replace with camera stream)
video_path = "videos/dad_walking_cropped.mp4" # Ensure this path is correct and the video is accessible
cap = cv2.VideoCapture(video_path)

frame_count = 0
detected_frames = []

# Create a directory to save detected faces, if it doesn't exist
output_dir = "detected_dad_faces"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Starting facial recognition...")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1

    try:
        result = DeepFace.verify(
            img1_path=reference_img_path, 
            img2_path=frame, 
            model_name="VGG-Face", 
            enforce_detection=True
        )

        if result["verified"]:
            print(f"Dad detected in frame {frame_count}!")
            detected_frames.append(frame_count)

            # Optionally, save the frame where your dad was detected
            output_frame_path = os.path.join(output_dir, f"dad_detected_frame_{frame_count}.jpg")
            cv2.imwrite(output_frame_path, frame)

    except Exception as e:
        print(f"Could not verify face in frame {frame_count}: {e}")
        continue

cap.release()
cv2.destroyAllWindows()
print("\nFinished processing.")
print(f"Dad was detected in a total of {len(detected_frames)} frames.")
print(f"Frames where dad was detected: {detected_frames}")

2025-06-16 11:44:51.777783: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-16 11:44:52.034427: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-16 11:44:52.159680: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750074292.421716   21135 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750074292.512853   21135 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750074292.626035   21135 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

Starting facial recognition...


2025-06-16 11:44:55.987579: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-06-16 11:44:56.262618: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2025-06-16 11:44:56.841341: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2025-06-16 11:44:57.095120: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 411041792 exceeds 10% of free system memory.
2025-06-16 11:44:57.428323: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.
2025-06-16 11:44:57.509568: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.


: 